# Homework 1 - Problem 2

Reload data from the open data portal

In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import matplotlib.pyplot as plt

client = Socrata("data.cityofchicago.org", None)
results = client.get("6zsd-86xi", where="year=2017 OR year=2018",limit=1000000)
results_df = pd.DataFrame.from_records(results)

Note: source consulted for geomerge https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html

Convert into geodataframe to be able to merge to census API
First, load necessary packages

In [2]:
import geopandas
import shapefile as shp
from shapely.geometry import Point

Create coordinate object to use as shapely object

In [3]:
results_df['latitude'] = pd.to_numeric(results_df['latitude'])
results_df['longitude'] = pd.to_numeric(results_df['longitude'])
results_df['Coordinates'] = list(zip(results_df.longitude, results_df.latitude))
results_df['Coordinates'] = results_df['Coordinates'].apply(Point)
gdf = geopandas.GeoDataFrame(results_df, geometry='Coordinates')

In [ ]:
#print(gdf.head())

Now, load ACS data via census API 
(source consulted - documentation)

In [4]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [5]:
import requests
request_obj = requests.get('https://api.census.gov/data/2017/acs/acs5/?get=NAME,B23025_001E,B17001_002M&for=block%20group:*&in=state:17%20county:031')
json = request_obj.json()
df_1 = pd.DataFrame(json)
df_1.columns = df_1.iloc[0]
df_1 = df_1.drop(df_1.index[0])
df_1.head()

,NAME,B23025_001E,B17001_002M,state,county,tract,block group
1,"Block Group 1, Census Tract 2519, Cook County,...",462,None,17,031,251900,1
2,"Block Group 2, Census Tract 2519, Cook County,...",899,None,17,031,251900,2
3,"Block Group 1, Census Tract 8211.01, Cook Coun...",1148,None,17,031,821101,1
4,"Block Group 3, Census Tract 8211.01, Cook Coun...",1035,None,17,031,821101,3
5,"Block Group 2, Census Tract 8211.01, Cook Coun...",556,None,17,031,821101,2


In [6]:
shp_path = "./tl_2018_17_bg.shp"
location_match = geopandas.read_file(shp_path)

In [13]:
location_match.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,17,175,951500,3,171759515003,Block Group 3,G5030,S,267762910,12295,+41.0283710,-089.8495082,"POLYGON ((-89.985479 40.977203, -89.9854879999..."
1,17,175,951400,4,171759514004,Block Group 4,G5030,S,23892876,0,+41.0998489,-089.8409271,"POLYGON ((-89.87840199999999 41.106688, -89.87..."
2,17,175,951400,3,171759514003,Block Group 3,G5030,S,156690063,322516,+41.1420518,-089.8638280,"POLYGON ((-89.98491399999999 41.084687, -89.98..."
3,17,175,951400,1,171759514001,Block Group 1,G5030,S,85399097,0,+41.2067522,-089.7212073,"POLYGON ((-89.810795 41.22781699999999, -89.81..."
4,17,183,011100,1,171830111001,Block Group 1,G5030,S,84897206,320477,+39.9921826,-087.5773378,"POLYGON ((-87.618442 39.999161, -87.6182269999..."


In [14]:
new_df = df_1.merge(location_match,how='left',left_on='tract',right_on='TRACTCE')

In [15]:
census_gdf = geopandas.GeoDataFrame(new_df, geometry='geometry')

In [19]:
census_gdf.shape

(15529, 20)

census_merge = geopandas.sjoin(test_gdf, census_gdf, how="inner", op='intersects')